# 说明

本例增加一个“**学习率**”的变化，即：随着训练epoch的增加，学习率不断下降！—— 希望tensorboard中增加learn_rate变量的展示！

方式：回调函数 + 自定义函数
- 创建文件编写器：tf.summary.create_file_writer() —— 把tf.summary.scalar()的记录的东西写到磁盘中！
- 自定义一个学习率变化功能函数；—— 为了给回调函数调用！
- 在学习率变化功能函数中，用tf.summary.scalar()记录学习率的变化；—— 为了往磁盘里存写！
- 来model.fit()中使用回调函数LearningRateScheduler() —— 总体的启动！

In [1]:
import tensorboard as tb
import tensorflow as tf

In [2]:
# 加载数据集：只用训练集即可
(train_image, train_label), (test_image, test_label) = tf.keras.datasets.mnist.load_data()

# 维度拓展，从数组转为图像尺寸
train_image = tf.expand_dims(train_image, -1)
test_image = tf.expand_dims(test_image, -1)
train_image.shape

# 转换数据类型：
train_image = tf.cast( train_image/255, tf.float32 )  # 归一化后，转为float32
train_label = tf.cast( train_label, tf.int32 )

test_image = tf.cast( test_image/255, tf.float32 )
test_label = tf.cast( test_label, tf.int32 )

In [3]:
# tf.data进行数据集输入：
train_dataset = tf.data.Dataset.from_tensor_slices( (train_image,train_label) )
test_dataset = tf.data.Dataset.from_tensor_slices( (test_image,test_label) )
train_dataset

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int32)>

In [4]:
# 数据集乱序、分批次：因为后面用tf.keras直接搭网络，所以训练数据一般要.repeat()
train_dataset = train_dataset.shuffle(60000).repeat().batch(64)
test_dataset = test_dataset.shuffle(10000).batch(64)

In [5]:
model = tf.keras.Sequential( [
    tf.keras.layers.Conv2D( 16, [3,3], activation = 'relu', input_shape = (None, None, 1) ),
    tf.keras.layers.Conv2D( 32, [3,3], activation = 'relu' ),
    tf.keras.layers.GlobalMaxPooling2D(),  # 换一个Max池化看看（不是重点）
    tf.keras.layers.Dense(10, activation = 'softmax')
] )

In [6]:
model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['acc']
)

### 创建回调函数：

In [7]:
import os
import datetime

In [8]:
# 自定义一个学习率变化功能函数：
def lr_rate(epoch):
    learning_rate = 0.2
    if epoch > 5:
        learning_rate = 0.02
    if epoch > 10:
        learning_rate = 0.01
    if epoch > 20:
        learning_rate = 0.005
    tf.summary.scalar( 'learning_rate', data = learning_rate, step = epoch )  # 时时记录
    return learning_rate

In [9]:
log_dir = os.path.join('logs', datetime.datetime.now().strftime("%Y%m%d_%H%M%S"))

# 文件编写器：把记录的东西写到磁盘中
file_writer = tf.summary.create_file_writer( log_dir + '/lr' )  # log_dir下现有3个文件夹：train、test、lr
file_writer.set_as_default()  # 设定为默认的文件编写器

In [10]:
# 创建回调函数：
tensorboard_callback = tf.keras.callbacks.TensorBoard( log_dir, histogram_freq = 1 )  
lr_callback = tf.keras.callbacks.LearningRateScheduler( lr_rate )  # 把上面写的函数，放入回调函数中

### 训练时使用：

In [ ]:
model.fit(
    train_dataset,
    epochs = 22,
    steps_per_epoch = 60000 // 64,
    validation_data = test_dataset,
    validation_steps = 10000 // 64,
    callbacks = [tensorboard_callback, lr_callback]
)

In [ ]:
# tensorboard在Notebook中显示的方法：
%load_ext tensorboard
%matplotlib inline

%tensorboard --logdir logs  # 其中logs就是那个文件夹名